In [1]:
import sys
sys.path.append("../../")
from padre.app import pypadre

In [2]:
pypadre.config.authenticate("hmafnan", "test")

In [3]:
list_of_datasets = pypadre.local_backend.datasets.list()
print("Datasets in local before deleting")
print(list_of_datasets)
for ds in list_of_datasets:
    pypadre.datasets.delete(ds)

print("Datasets in local")
print(pypadre.local_backend.datasets.list())

print("Datasets on server")
print(pypadre.remote_backend.datasets.list())

Datasets in local before deleting
Datasets in local
[]
Datasets on server
[]


In [4]:
# Download and save dataset from openMl
name = "diabetes"
downloads = pypadre.datasets.search_downloads(name)
print("Datasets available on openMl for " + name)
print(downloads)
print("Saving OpenMl Datasets... ")
for externel_dataset in pypadre.datasets.download(downloads):
    pypadre.datasets.put(externel_dataset, upload=True)

Datasets available on openMl for diabetes
[{'did': 37, 'name': 'diabetes', 'format': 'ARFF', 'status': 'active', 'MajorityClassSize': 500, 'MaxNominalAttDistinctValues': 2, 'MinorityClassSize': 268, 'NumberOfClasses': 2, 'NumberOfFeatures': 9, 'NumberOfInstances': 768, 'NumberOfInstancesWithMissingValues': 0, 'NumberOfMissingValues': 0, 'NumberOfNumericFeatures': 8, 'NumberOfSymbolicFeatures': 1}]
Saving OpenMl Datasets... 
Uploading dataset diabetes, (768, 9), Multivariat


In [5]:
print("Total available datasets in Padre")
for ds in pypadre.datasets.list():
    pypadre.datasets.print_dataset_details(ds)

Total available datasets in Padre
Metadata for dataset 1
	id=f8f34812-2932-4498-8a1f-d9d704f50227
	name=diabetes
	version=1
	description=**Author**: [Vincent Sigillito](vgs@aplcen.apl.jhu.edu)  

**Source**: [Obtained from UCI](https://archive.ics.uci.edu/ml/datasets/pima+indians+diabetes) 

**Please cite**: [UCI citation policy](https://archive.ics.uci.edu/ml/citation_policy.html)  

1. Title: Pima Indians Diabetes Database
 
 2. Sources:
    (a) Original owners: National Institute of Diabetes and Digestive and
                         Kidney Diseases
    (b) Donor of database: Vincent Sigillito (vgs@aplcen.apl.jhu.edu)
                           Research Center, RMI Group Leader
                           Applied Physics Laboratory
                           The Johns Hopkins University
                           Johns Hopkins Road
                           Laurel, MD 20707
                           (301) 953-6231
    (c) Date received: 9 May 1990
 
 3. Past Usage:
     1. Smith,~J

In [6]:
# OpenMl dataset which is uploaded to server
did = "1"
print("Datasets %s on server" % did)
ds = pypadre.remote_backend.datasets.get(did)
print(ds)

Datasets 1 on server
1_diabetes: http://www.padre-lab.eu/onto/Multivariat, (768, 9), pandas


In [7]:
# Run experiment with openML dataset
import uuid
import pprint
from padre.core import Experiment

def create_test_pipeline():
    from sklearn.pipeline import Pipeline
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    # estimators = [('reduce_dim', PCA()), ('clf', SVC())]
    estimators = [('SVC', SVC(probability=True))]
    return Pipeline(estimators)

def split(idx):
    # Do a 70:30 split
    limit = int(.7 * len(idx))
    return idx[0:limit], idx[limit:], None

test_experiment_name = "Test Experiment SVM " + str(uuid.uuid4())[0:9] # Unique name for experiment
ex = Experiment(name=test_experiment_name,
                    description="Testing Support Vector Machines via SKLearn Pipeline",
                    dataset=ds,
                    workflow=create_test_pipeline(), keep_splits=True, strategy="random",
                    function=split)
conf = ex.configuration()  # configuration, which has been automatically extracted from the pipeline
pprint.pprint(ex.hyperparameters())  # get and print hyperparameters
ex.execute()  # run the experiment and report

pypadre.metrics_evaluator.add_experiments([ex, ex])

{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'c-oja pas vector classification'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'C-Support Vector Classification.\n'
                             '\n'
                             '    The implementation is based on libsvm. The '
                             'fit time complexity\n'
                             '    is more than quadratic with the number of '
                             'samples which makes it hard\n'
                             '    to scale to dataset with more than a couple '
                             'of 10000 samples.\n'
                             '\n'
                             '    The multiclass support is handled according '
                             'to a one-vs-one scheme.\n'
                             '\n'
                             '    For details on the precise mathematical '
            

INFO: 2019-03-06 14:50:40.935 Experiment<id:Test Experiment SVM 1efa645d->:	start: phase=experiment
INFO: 2019-03-06 14:50:41.473 	Run<id:137a8585-d17f-4a89-a98e-ade5c060a404;name:Test Experiment SVM 1efa645d->:	start: phase=run
INFO: 2019-03-06 14:50:42.039 		Split<id:808ce6e0-2bcf-41f7-910b-1cb6917063d2;name:Test Experiment SVM 1efa645d->:	start: phase=split
INFO: 2019-03-06 14:50:42.586 		Split<id:808ce6e0-2bcf-41f7-910b-1cb6917063d2;name:Test Experiment SVM 1efa645d->:	stop: phase=split
INFO: 2019-03-06 14:50:42.588 	Run<id:137a8585-d17f-4a89-a98e-ade5c060a404;name:Test Experiment SVM 1efa645d->:	stop: phase=run
INFO: 2019-03-06 14:50:42.589 Experiment<id:Test Experiment SVM 1efa645d->:	stop: phase=experiment


In [8]:
# Get experiment from server
#experiments = pypadre.remote_backend.experiments.list_experiments(test_experiment_name)
#print(experiments)
ex = pypadre.remote_backend.experiments.get_experiment("1")
print(ex)

WARN: 2019-03-06 14:50:56.695 <padre.experimentcreator.ExperimentCreator object at 0x7f392cc99438>:	max_iter not present in list for estimator:c-oja pas vector classification
WARN: 2019-03-06 14:50:56.695 <padre.experimentcreator.ExperimentCreator object at 0x7f392cc99438>:	cache_size not present in list for estimator:c-oja pas vector classification
WARN: 2019-03-06 14:50:56.696 <padre.experimentcreator.ExperimentCreator object at 0x7f392cc99438>:	max_iter not present in list for estimator:c-oja pas vector classification
WARN: 2019-03-06 14:50:56.696 <padre.experimentcreator.ExperimentCreator object at 0x7f392cc99438>:	cache_size not present in list for estimator:c-oja pas vector classification
INFO: 2019-03-06 14:50:56.696 <padre.experimentcreator.ExperimentCreator object at 0x7f392cc99438>:	Test Experiment SVM 1efa645d- created successfully!
{'description': 'Testing Support Vector Machines via SKLearn Pipeline', 'dataset': ['diabetes'], 'workflow': ['SVC'], 'strategy': 'random', 'par

<padre.experimentcreator.ExperimentCreator object at 0x7f392cc99438>:	max_iter not present in list for estimator:c-oja pas vector classification
<padre.experimentcreator.ExperimentCreator object at 0x7f392cc99438>:	cache_size not present in list for estimator:c-oja pas vector classification
<padre.experimentcreator.ExperimentCreator object at 0x7f392cc99438>:	max_iter not present in list for estimator:c-oja pas vector classification
<padre.experimentcreator.ExperimentCreator object at 0x7f392cc99438>:	cache_size not present in list for estimator:c-oja pas vector classification


In [1]:
# Import PadreApp from a package
from padre.app import pypadre
# All interactions with padre should be done via the app

[WARNING] [12:32:26:pypadre - http] Authentication token is NONE. You need to authentication for user mgrani with your current password (or set a new user)


In [2]:
# Configuration of padreapp via a configuration file including backend, http, server authentication etc
"""
 from padre.app.padre_app import pypadre
 assert pypadre.offline
 pypadre.authenticate("test", "mgrani") # authentication sets the pypadre app into online mode
 pypadre.config.save()
 assert pypadre.offline == False
 # delete all datasets
 for ds in pypadre.datasets.list():
     pypadre.datasets.delete(ds)
 assert len([ds for ds in pypadre.datasets.list()])==0 # after deletion there should be no datasets
 # fetch downloads. This should include the datasets that are currently supported.
 downloads = pypadre.datasets.search_downloads("Iris", type="multivariate") # search for external datasets
 # user can manipulate downloads here
 downloaded = pypadre.datasets.download(downloads)
 for download in downloaded: 
    pypadre.datasets.put(ds)
 available = pypadre.datasets.list("Iris")
 assert downloads.equals(available) # the check might be more complex here, but both sets should be equal
 dataset = pypadre.datasets.get(available[0])
"""

'\n from padre.app.padre_app import pypadre\n assert pypadre.offline\n pypadre.authenticate("test", "mgrani") # authentication sets the pypadre app into online mode\n pypadre.config.save()\n assert pypadre.offline == False\n # delete all datasets\n for ds in pypadre.datasets.list():\n     pypadre.datasets.delete(ds)\n assert len([ds for ds in pypadre.datasets.list()])==0 # after deletion there should be no datasets\n # fetch downloads. This should include the datasets that are currently supported.\n downloads = pypadre.datasets.search_downloads("Iris", type="multivariate") # search for external datasets\n # user can manipulate downloads here\n downloaded = pypadre.datasets.download(downloads)\n for download in downloaded: \n    pypadre.datasets.put(ds)\n available = pypadre.datasets.list("Iris")\n assert downloads.equals(available) # the check might be more complex here, but both sets should be equal\n dataset = pypadre.datasets.get(available[0])\n'

In [3]:
# List all available datasets(Toy Datasets)
print(pypadre.datasets.list())

[]


In [4]:
# List all currently available local experiments
print(pypadre.experiments.list_experiments())

['Decorated Experiment', 'Test Experiment SVM', 'Grid_search_experiment_4', 'Test Experiment 2', 'iPython Experiment1 ', 'Test Experiment PCA Logistic(Iris)', 'Grid_search_experiment_3', 'JSON FILE TEST EXPERIMENT', 'IPython Experiment 3', 'Test Experiment PCA Logistic(Digits)', 'Executor Test 1(Boston_House_Prices)', 'Test Experiment PCA Logistic(Breast_Cancer)', 'Executor Test 3', 'Grid_search_experiment_1', 'Sacred_Experiment_Test', 'Torch', 'Executor Test 2', 'Grid_search_experiment_2', 'Executor Test 1(Diabetes)', 'Grid_search_experiment_5', 'Executor Test 1(Iris)']


In [5]:
# Create a simple experiment via experiment creator utilizing a dataset displayed. Run it with multiple parameters
pypadre.experiment_creator.clear_experiments()
workflow = pypadre.experiment_creator.create_test_pipeline(['isomap embedding'])
pypadre.experiment_creator.create(name='iPython Experiment1 ', 
                                           description='Simple iPython notebook experiment',
                                           workflow=workflow, 
                                           dataset_list=['Digits'])
pypadre.experiment_creator.execute()

# Aimed at new users for running already created experiments or running from JSON files

INFO: 2019-02-06 12:32:28.960 <padre.experimentcreator.ExperimentCreator object at 0x7f34ab763860>:	iPython Experiment1  created successfully!
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n'
                             '\n'
                             '    Read more in the :ref:`User Guide '
                             '<isomap>`.\n'
                             '\n'
                             '    Parameters\n'
                             '    ----------\n'
                             '    n_neighbors : integer\n'
                             '        number of neighbors to consider for each '
                             'poin

In [6]:
# Using decorators to run experiments
from padre.ds_import import load_sklearn_toys
from padre import *
from padre.app import pypadre
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

@Workflow(exp_name="Decorated Experiment",
          description="Test experiment with decorators and SVC")
def create_test_pipeline():
    estimators = [('clf', SVC(probability=True))]
    return Pipeline(estimators)


@Dataset(exp_name="Decorated Experiment")
def get_dataset():
    ds = [i for i in load_sklearn_toys()][2]
    return ds



In [7]:
ex = pypadre.experiments.run(decorated=True)
ex = run("Decorated Experiment")  # run the experiment and report
# Aimed at engineers and scientists for quick prototyping and trying out features

creating the workflow
INFO: 2019-02-06 12:32:32.131 Experiment<id:Decorated Experiment>:	start: phase=experiment
INFO: 2019-02-06 12:32:32.132 	Run<id:e46027ae-2a02-11e9-b5a1-1860248c6d22;name:Decorated Experiment>:	start: phase=run
INFO: 2019-02-06 12:32:32.132 		Split<id:0_e46027af-2a02-11e9-b5a1-1860248c6d22;name:Decorated Experiment>:	start: phase=split
INFO: 2019-02-06 12:32:32.613 		Split<id:0_e46027af-2a02-11e9-b5a1-1860248c6d22;name:Decorated Experiment>:	stop: phase=split
INFO: 2019-02-06 12:32:32.624 	Run<id:e46027ae-2a02-11e9-b5a1-1860248c6d22;name:Decorated Experiment>:	stop: phase=run
INFO: 2019-02-06 12:32:32.636 Experiment<id:Decorated Experiment>:	stop: phase=experiment
creating the workflow
INFO: 2019-02-06 12:32:32.706 Experiment<id:Decorated Experiment>:	start: phase=experiment
INFO: 2019-02-06 12:32:32.707 	Run<id:e46027b0-2a02-11e9-b5a1-1860248c6d22;name:Decorated Experiment>:	start: phase=run
INFO: 2019-02-06 12:32:32.708 		Split<id:0_e46027b1-2a02-11e9-b5a1-18602

In [8]:
# Creating and running experiments via pure code (example.py)
def create_test_pipeline():
    from sklearn.pipeline import Pipeline
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    # estimators = [('reduce_dim', PCA()), ('clf', SVC())]
    estimators = [('clf', SVC(probability=True))]
    return Pipeline(estimators)



In [9]:
from padre.ds_import import load_sklearn_toys
from padre.core import Experiment
from padre.base import PadreLogger
from padre.eventhandler import add_logger

logger = PadreLogger()
logger.backend = pypadre.repository
add_logger(logger=logger)
ds = [i for i in load_sklearn_toys()][2]
ex = Experiment(name="Test Experiment SVM",
                    description="Testing Support Vector Machines via SKLearn Pipeline",
                    dataset=ds,
                    workflow=create_test_pipeline(), keep_splits=True, strategy="random")
ex.execute()

INFO: 2019-02-06 12:32:36.051 Experiment<id:Test Experiment SVM>:	start: phase=experiment
INFO: 2019-02-06 12:32:36.051 Experiment<id:Test Experiment SVM>:	start: phase=experiment
INFO: 2019-02-06 12:32:36.052 	Run<id:e6b656ea-2a02-11e9-b5a1-1860248c6d22;name:Test Experiment SVM>:	start: phase=run
INFO: 2019-02-06 12:32:36.053 	Run<id:e6b656ea-2a02-11e9-b5a1-1860248c6d22;name:Test Experiment SVM>:	start: phase=run
INFO: 2019-02-06 12:32:36.053 		Split<id:0_e6b656eb-2a02-11e9-b5a1-1860248c6d22;name:Test Experiment SVM>:	start: phase=split
INFO: 2019-02-06 12:32:36.054 		Split<id:0_e6b656eb-2a02-11e9-b5a1-1860248c6d22;name:Test Experiment SVM>:	start: phase=split
INFO: 2019-02-06 12:32:36.532 		Split<id:0_e6b656eb-2a02-11e9-b5a1-1860248c6d22;name:Test Experiment SVM>:	stop: phase=split
INFO: 2019-02-06 12:32:36.543 		Split<id:0_e6b656eb-2a02-11e9-b5a1-1860248c6d22;name:Test Experiment SVM>:	stop: phase=split
INFO: 2019-02-06 12:32:36.554 	Run<id:e6b656ea-2a02-11e9-b5a1-1860248c6d22;name:

In [10]:
# The PyTorch wrapper supports the use of User Defined Callbacks 
from padre.core.wrappers.wrapper_pytorch import WrapperPytorch, CallBack
class TestCallbacks(CallBack):
    def on_compute_loss(self, loss):
        print('Function on compute loss. Loss = {loss}'.format(loss=loss))

    def on_epoch_end(self, obj):
        print('Epoch ended')

    def on_epoch_start(self, obj):
        print('Epoch started')

    def on_iteration_start(self, obj):
        print('Iteration started')

    def on_iteration_end(self, obj):
        print('Iteration ended')

In [11]:
# Running an experiment with a custom wrapper
from sklearn.pipeline import Pipeline

# Define the parameters
params = dict()

import json
with open('../PyPaDRe/tests/proof_of_concept/Pytorch/classification.json') as json_data:
    params = json.load(json_data)

obj = WrapperPytorch(params=params)
obj.set_callbacks([TestCallbacks()])
estimators = [('clf', obj)]
workflow = Pipeline(estimators)
ds = [i for i in load_sklearn_toys()][4]

ex = Experiment(name="Torch",
                description="Testing Torch via SKLearn Pipeline",
                dataset=ds,
                workflow=workflow)
ex.execute()

INFO: 2019-02-06 12:32:41.217 Experiment<id:Torch>:	start: phase=experiment
INFO: 2019-02-06 12:32:41.218 Experiment<id:Torch>:	start: phase=experiment
INFO: 2019-02-06 12:32:41.219 	Run<id:e9cab7d6-2a02-11e9-b5a1-1860248c6d22;name:Torch>:	start: phase=run
INFO: 2019-02-06 12:32:41.220 	Run<id:e9cab7d6-2a02-11e9-b5a1-1860248c6d22;name:Torch>:	start: phase=run
INFO: 2019-02-06 12:32:41.220 		Split<id:0_e9cab7d7-2a02-11e9-b5a1-1860248c6d22;name:Torch>:	start: phase=split
INFO: 2019-02-06 12:32:41.221 		Split<id:0_e9cab7d7-2a02-11e9-b5a1-1860248c6d22;name:Torch>:	start: phase=split
Epoch started
Iteration started
Function on compute loss. Loss = 0.27330637689264087
1 0.27330637689264087
Iteration ended
Iteration started
Function on compute loss. Loss = 0.28438050996071457
2 0.28438050996071457
Iteration ended
Iteration started
Function on compute loss. Loss = 0.21685942438453013
3 0.21685942438453013
Iteration ended
Iteration started
Function on compute loss. Loss = 0.1779247002048193
4 0

/home/christofer/PycharmProjects/test_folder/venv/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [14]:
# Compare the metrics between experiments and also between runs(Displaying pandas dataframe)
from padre.experimentexecutor import ExperimentExecutor
# Create an experiment with multiple runs
params_pca = {'num_components': [2, 3, 4, 5, 6]}
params_svr = {'C': [0.5, 1.0, 1.5],
              'poly_degree': [1, 2, 3]}
params_dict = {'SVR': params_svr, 'pca': params_pca}
workflow = pypadre.experiment_creator.create_test_pipeline(['pca', 'SVR'])
params_dict = pypadre.experiment_creator.convert_alternate_estimator_names(params_dict)
pypadre.experiment_creator.clear_experiments('IPython Experiment 3')
pypadre.experiment_creator.create(name='IPython Experiment 3',
                                  description='Experiment to test the comparison of metrics',
                                  dataset_list=['Boston_House_Prices'],
                                  workflow=workflow,
                                  params=params_dict
                                 )
experiments_executor = ExperimentExecutor(experiments=pypadre.experiment_creator.createExperimentList())
experiments_executor.execute(local_run=True, threads=1)
#from padre.metrics import CompareMetrics
#metrics = CompareMetrics(experiments_list=experiments_executor.experiments)
#print(metrics.show_metrics())
pypadre.metrics_evaluator.add_experiments(experiments_executor.experiments)
print(pypadre.metrics_evaluator.show_metrics())

INFO: 2019-02-06 12:33:14.058 <padre.experimentcreator.ExperimentCreator object at 0x7f34ab763860>:	IPython Experiment 3 created successfully!
INFO: 2019-02-06 12:33:14.058 <padre.experimentcreator.ExperimentCreator object at 0x7f34ab763860>:	IPython Experiment 3 created successfully!
Sequential Execution
INFO: 2019-02-06 12:33:14.147 <padre.experimentexecutor.ExperimentExecutor object at 0x7f3471268128>:	Executing experiment: iPython Experiment1 
INFO: 2019-02-06 12:33:14.147 <padre.experimentexecutor.ExperimentExecutor object at 0x7f3471268128>:	Executing experiment: iPython Experiment1 
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n

INFO: 2019-02-06 12:33:14.275 	Run<id:fd764822-2a02-11e9-b5a1-1860248c6d22;name:iPython Experiment1 >:	start: phase=run
INFO: 2019-02-06 12:33:14.277 		Split<id:0_fd764823-2a02-11e9-b5a1-1860248c6d22;name:iPython Experiment1 >:	start: phase=split
INFO: 2019-02-06 12:33:14.278 		Split<id:0_fd764823-2a02-11e9-b5a1-1860248c6d22;name:iPython Experiment1 >:	start: phase=split
INFO: 2019-02-06 12:33:15.006 		Split<id:0_fd764823-2a02-11e9-b5a1-1860248c6d22;name:iPython Experiment1 >:	stop: phase=split
INFO: 2019-02-06 12:33:15.006 		Split<id:0_fd764823-2a02-11e9-b5a1-1860248c6d22;name:iPython Experiment1 >:	stop: phase=split
INFO: 2019-02-06 12:33:15.006 	Run<id:fd764822-2a02-11e9-b5a1-1860248c6d22;name:iPython Experiment1 >:	stop: phase=run
INFO: 2019-02-06 12:33:15.006 	Run<id:fd764822-2a02-11e9-b5a1-1860248c6d22;name:iPython Experiment1 >:	stop: phase=run
INFO: 2019-02-06 12:33:15.007 Experiment<id:iPython Experiment1 >:	stop: phase=experiment
INFO: 2019-02-06 12:33:15.007 Experiment<id:iP

INFO: 2019-02-06 12:33:15.207 		Split<id:0_fd764829-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-06 12:33:15.208 		Split<id:0_fd764829-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-06 12:33:15.208 	Run<id:fd764828-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-06 12:33:15.208 	Run<id:fd764828-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-06 12:33:15.210 	Run<id:fd76482a-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-06 12:33:15.211 	Run<id:fd76482a-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-06 12:33:15.211 		Split<id:0_fd76482b-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-06 12:33:15.212 		Split<id:0_fd76482b-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-06

INFO: 2019-02-06 12:33:15.422 		Split<id:0_fd764843-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-06 12:33:15.422 		Split<id:0_fd764843-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-06 12:33:15.423 	Run<id:fd764842-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-06 12:33:15.423 	Run<id:fd764842-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-06 12:33:15.426 	Run<id:fd764844-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-06 12:33:15.426 	Run<id:fd764844-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-06 12:33:15.427 		Split<id:0_fd764845-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-06 12:33:15.427 		Split<id:0_fd764845-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-06

INFO: 2019-02-06 12:33:15.623 	Run<id:fd76485c-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-06 12:33:15.623 	Run<id:fd76485c-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-06 12:33:15.624 		Split<id:0_fd76485d-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-06 12:33:15.625 		Split<id:0_fd76485d-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-06 12:33:15.638 		Split<id:0_fd76485d-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-06 12:33:15.639 		Split<id:0_fd76485d-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-06 12:33:15.639 	Run<id:fd76485c-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-06 12:33:15.639 	Run<id:fd76485c-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-06

INFO: 2019-02-06 12:33:15.823 		Split<id:0_fd764875-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-06 12:33:15.824 		Split<id:0_fd764875-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-06 12:33:15.836 		Split<id:0_fd764875-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-06 12:33:15.836 		Split<id:0_fd764875-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-06 12:33:15.837 	Run<id:fd764874-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-06 12:33:15.837 	Run<id:fd764874-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-06 12:33:15.839 	Run<id:fd764876-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-06 12:33:15.840 	Run<id:fd764876-2a02-11e9-b5a1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-06

In [16]:
# Analyze metrics where the runs have specific values for a component
pypadre.metrics_evaluator.analyze_runs(
    ['principal component analysis.num_components.4', 'principal component analysis.num_components.5'])
df = pypadre.metrics_evaluator.display_results()
print(df)

                                     run  \
0   fd764850-2a02-11e9-b5a1-1860248c6d22   
1   fd76485a-2a02-11e9-b5a1-1860248c6d22   
2   fd764870-2a02-11e9-b5a1-1860248c6d22   
3   fd764834-2a02-11e9-b5a1-1860248c6d22   
4   fd764848-2a02-11e9-b5a1-1860248c6d22   
5   fd76482a-2a02-11e9-b5a1-1860248c6d22   
6   fd764832-2a02-11e9-b5a1-1860248c6d22   
7   fd764852-2a02-11e9-b5a1-1860248c6d22   
8   fd764846-2a02-11e9-b5a1-1860248c6d22   
9   fd764878-2a02-11e9-b5a1-1860248c6d22   
10  fd76483e-2a02-11e9-b5a1-1860248c6d22   
11  fd764866-2a02-11e9-b5a1-1860248c6d22   
12  fd76483c-2a02-11e9-b5a1-1860248c6d22   
13  fd764828-2a02-11e9-b5a1-1860248c6d22   
14  fd76487a-2a02-11e9-b5a1-1860248c6d22   
15  fd76485c-2a02-11e9-b5a1-1860248c6d22   
16  fd764864-2a02-11e9-b5a1-1860248c6d22   
17  fd76486e-2a02-11e9-b5a1-1860248c6d22   

                                     split                      dataset  \
0   0_fd764851-2a02-11e9-b5a1-1860248c6d22  Boston House Prices dataset   
1   0_fd76485

In [17]:
# Display only the metric specified by the user
pypadre.metrics_evaluator.analyze_runs(['principal component analysis.num_components.4'], ['mean_error'])
df = pypadre.metrics_evaluator.display_results()
print(df)

                                    run  \
0  fd764850-2a02-11e9-b5a1-1860248c6d22   
1  fd76485a-2a02-11e9-b5a1-1860248c6d22   
2  fd764832-2a02-11e9-b5a1-1860248c6d22   
3  fd764846-2a02-11e9-b5a1-1860248c6d22   
4  fd764878-2a02-11e9-b5a1-1860248c6d22   
5  fd76483c-2a02-11e9-b5a1-1860248c6d22   
6  fd764828-2a02-11e9-b5a1-1860248c6d22   
7  fd764864-2a02-11e9-b5a1-1860248c6d22   
8  fd76486e-2a02-11e9-b5a1-1860248c6d22   

                                    split                      dataset  \
0  0_fd764851-2a02-11e9-b5a1-1860248c6d22  Boston House Prices dataset   
1  0_fd76485b-2a02-11e9-b5a1-1860248c6d22  Boston House Prices dataset   
2  0_fd764833-2a02-11e9-b5a1-1860248c6d22  Boston House Prices dataset   
3  0_fd764847-2a02-11e9-b5a1-1860248c6d22  Boston House Prices dataset   
4  0_fd764879-2a02-11e9-b5a1-1860248c6d22  Boston House Prices dataset   
5  0_fd76483d-2a02-11e9-b5a1-1860248c6d22  Boston House Prices dataset   
6  0_fd764829-2a02-11e9-b5a1-1860248c6d22  Boston H

In [27]:
# Go online, register new user
# For registering, user has to use the browser to enter details

In [20]:
# Log in to server
# Implemented

In [21]:
# Upload/Sync experiments on demand
# Not Implemented. Experiments can only be uploaded during the execution of the experiment

In [22]:
# Look at the experiment via SPA and check the results and visualize them(if possible)

In [23]:
# Download an experiment from the server and run it.(via the experiment.JSON file) Example for reproducibility
# Not yet implemented

In [ ]:
# Keep track of random seeds used in the experiment
# Not yet implemented

In [ ]:
# Download a dataset from OpenML and upload it to the server
# Generate metrics for uploaded  datasets
# Currently in progress. Some datasets in OpenML do not work and raise exceptions

In [ ]:
# Download a dataset from server
# Implemented

In [ ]:
# Running an experiment with a dataset downloaded from the server and sync the experiment automatically
# Not implemented

In [ ]:
# Running an experiment on the server cluster

In [ ]:
# Tracking the progress of experiments including hyperparameter search
# Code to be included in the backend for logging the porogress

In [ ]:
# Visualizing the results of an experiment/ dataset